# Lab: Introduction to Neural Networks in PyTorch

**Duration:** 3 Hours
**Topic:** Tensors, Linear Regression, and Non-Linear Neural Networks (MLPs)

## 1. Introduction
In this laboratory, you will explore PyTorch tensors, build a linear regression model, and finally construct a Multi-Layer Perceptron (MLP) to solve a non-linear classification problem.

**Learning Objectives:**
1. Manipulate PyTorch tensors.
2. Build custom `nn.Module` classes.
3. Understand why non-linearity (ReLU) is essential.

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split
import numpy as np

print(f'PyTorch version: {torch.__version__}')

## Part 1: Tensors (The Building Blocks)
Tensors are multi-dimensional arrays similar to NumPy arrays but can run on GPUs.

### Exercise 1: Creating Tensors
**Task:**
1. Create a random tensor with shape `(5, 3)` using `torch.rand`.
2. Create a tensor of all ones with shape `(2, 2)`.
3. Print their shapes and data types.

### Exercise 2: Matrix Multiplication & Shapes
Matrix multiplication is strict about shapes. Inner dimensions must match: `(A, B) @ (B, C)` works.
**Task:** Fix the error in the code below using a transpose `.T` operation.

In [ ]:
tensor_A = torch.tensor([[1, 2], [3, 4], [5, 6]]) # Shape (3, 2)
tensor_B = torch.tensor([[7, 10], [8, 11], [9, 12]]) # Shape (3, 2)

# TODO: Fix this line so it runs


## Part 2: Linear Regression Data
We will create synthetic data for a linear regression problem: $y = weight * X + bias$.

1.  **Define Parameters:** Set `weight = 0.7` and `bias = 0.3`.
2.  **Generate Data:**
    * Create a tensor `X` with values from `0` to `1` with a step of `0.02`.
    * Create a tensor `y` using the linear regression formula: $y = \text{weight} \times X + \text{bias}$.
3.  **Split Data:**
    * Calculate the split index for an **80/20** split (80% training, 20% testing).
    * Create `X_train`, `y_train`, `X_test`, and `y_test` based on this split.
4.  **Verify:** Print the number of samples in the training and testing sets to confirm the split.

### Exercise 3: Visualization
**Task:** Run the helper function below to visualize the linear data.

In [ ]:
def plot_predictions(train_data=X_train, train_labels=y_train, 
                     test_data=X_test, test_labels=y_test, predictions=None):
    plt.figure(figsize=(10, 7))
    plt.scatter(train_data, train_labels, c='b', s=4, label='Training data')
    plt.scatter(test_data, test_labels, c='g', s=4, label='Testing data')
    if predictions is not None:
        plt.scatter(test_data, predictions, c='r', s=4, label='Predictions')
    plt.legend(prop={'size': 14})
    plt.show()

plot_predictions()

## Part 3: Building a Linear Model
**Exercise 4:** Define a subclass of `nn.Module` called `LinearRegressionModel`. Use `nn.Linear` in the constructor.

In [ ]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        # TODO: Define a linear layer: in_features=1, out_features=1


    def forward(....) -> torch.Tensor:
        # TODO: Implement forward pass
        return 

torch.manual_seed(42)
model_0 = LinearRegressionModel()

## Part 4: Training
**Exercise 5:** Setup Loss and Optimizer. Use `nn.L1Loss` (MAE) and `torch.optim.SGD`.

In [ ]:
# TODO: Define Loss and Optimizer


**Exercise 6:** Complete the training loop.

In [ ]:
epochs = 100
for epoch in range(epochs):
    model_0.train()
    # 1. Forward pass
    # 2. Calculate loss
    # 3. Zero gradients
    # 4. Backpropagation
    # 5. Step optimizer
    
    if epoch % 10 == 0:
        print(f'Epoch: {epoch} | Loss: {loss.item()}')

**Exercise 7:** Prediction. Make predictions on `X_test` using `torch.inference_mode()`.

In [ ]:
#TODO make preds

plot_predictions(predictions=y_preds)

## Part 6: Non-Linear Data & Neural Networks (MLP)
Linear models cannot separate concentric circles. We need a neural network with **non-linear activation functions** (like ReLU) to bend the decision boundary.

In [ ]:
# Generate non-linear data
n_samples = 1000
X_circles, y_circles = make_circles(n_samples, noise=0.03, random_state=42)

# Convert to tensors

# Split data

# Visualize


### Exercise 8: Building a Non-Linear Model (MLP)
**Task:** Create a class `CircleModel` that subclasses `nn.Module`.
1. In `__init__`, define: 
   - `layer_1`: Linear (input 2 -> hidden 10)
   - `relu`: ReLU activation
   - `layer_2`: Linear (hidden 10 -> output 1)
2. In `forward`, connect them: `x -> layer_1 -> relu -> layer_2 -> output`

In [ ]:
class CircleModel(nn.Module):
    def __init__(self):
        # TODO: Define layers here

    def forward(self, x):
        # TODO: Implement forward pass

model_1 = CircleModel()
print(model_1)

### Exercise 9: Training the Classifier
**Task:** Train the model for 1000 epochs.
1. Loss Function: Use `nn.BCEWithLogitsLoss()` (Binary Cross Entropy for classification).
2. Optimizer: `SGD` with `lr=0.1`.
3. Loop: Forward pass -> Calculate Loss -> Zero Grad -> Backward -> Step.
*Hint: The model outputs 'logits'. To get accuracy, convert logits to labels.*

In [ ]:
epochs = 1000
for epoch in range(epochs):
    model_1.train()
    
    # 1. Forward pass
    
    # 2. Calculate loss
    
    # 3. Optimization step

    
    if epoch % 100 == 0:
        print(f'Epoch: {epoch} | Loss: {loss.item():.5f}')